In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
PROJECT_ID = "itg-bpma-gbl-ww-np"  # @param {type:"string"}
REGION = "europe-west1" 

In [4]:
bq_client = bigquery.Client(
    project=PROJECT_ID,  # GCP project used for running the queries and billing
)

In [5]:

def import_true_promo(client, zero_percent, month, num_weeks,channel=None, fill_discontinuity=False, keep_non_promo=False):
    def query(zero_percent, keep_non_promo = False):
        

        if keep_non_promo:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                WHERE
                    td.end_date >= (SELECT min_date FROM MinPromoDate)
                """.format(zero_percent)
        else:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                ),
                PromoFilter AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN is_promo = TRUE THEN 1 ELSE 0 END) > 0 AS has_promo
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        has_promo
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                JOIN
                    PromoFilter pf
                ON
                    td.ean = pf.ean
                    AND td.global_channel_type = pf.global_channel_type
                WHERE
                    td.end_date >= (SELECT min_date FROM MinPromoDate)
                """.format(zero_percent)
        if channel=='Online': 
            a+="""AND td.global_channel_type = 'Online'
            ORDER BY
                td.end_date;"""
        elif channel=='Offline':
            a+="""AND td.global_channel_type = 'Offline'
            ORDER BY
                td.end_date;"""
        else:
            a+="""
            ORDER BY
                td.end_date;"""
        return a

    data =client.query_and_wait(query(zero_percent, keep_non_promo)).to_dataframe()
    print("number of products before preprocessing", data["ean"].unique().shape[0])
    # Step 1: Count unique end dates for each EAN and each global channel type
    unique_dates = data.groupby(['ean', 'global_channel_type'])['end_date'].nunique().reset_index()

    # Step 2: Filter to find EANs where each global channel type has more than 65 unique dates
    eans_77_dates = unique_dates[unique_dates['end_date'] >= num_weeks]
    valid_eans = eans_77_dates.groupby('ean').filter(lambda x: len(x) == data["global_channel_type"].unique().shape[0] and all(x['end_date'] >= num_weeks))

    # Step 3: Filter the original DataFrame to include only these EANs
    data = data[data['ean'].isin(valid_eans['ean'])]
    data["sold_units"] = data["sold_units"].astype(float)
    data = data.sort_values(by=["end_date", "global_channel_type", "ean"])

    data['ean_global_channel'] = data['ean'] + '_' + data['global_channel_type']
    data['sub_tactic'] = data['sub_tactic'].str.lower().str.strip()

    def aggregate_subtactics(series):
        if series is None or all(pd.isnull(series)): 
            return ''
        all_subtactics = set()
        for items in series.dropna():
            tactics = set(item.strip() for item in items.split(','))
            all_subtactics.update(tactics)
        return ', '.join(sorted(all_subtactics))

    def custom_price_range(series):
        return series.mean(skipna=True) if not series.isnull().all() else np.nan

    aggregated_data = data.groupby(['start_date', 'end_date', 'ean_global_channel']).agg({
        'is_promo': 'first',
        'price_range': custom_price_range,
        'sub_tactic': aggregate_subtactics,
        'sub_axis': 'first',
        'seasonality_index': 'first',
        'sold_units': 'first'
    }).reset_index()

    aggregated_data.drop_duplicates(inplace=True)
    print("How many ean_global_channel_type:", aggregated_data.ean_global_channel.unique().shape[0])
    one_hot_encoded_data = aggregated_data['sub_tactic'].str.get_dummies(', ')
    empty_sub_tactic_indices = aggregated_data[aggregated_data['sub_tactic'] == ''].index
    one_hot_encoded_data.loc[empty_sub_tactic_indices] = 0

    final_data = pd.concat([aggregated_data, one_hot_encoded_data], axis=1)
    final_data.drop(['sub_tactic'], axis=1, inplace=True)

    def shuffle_and_sort(group):
        shuffled_group = group.sample(frac=1).reset_index(drop=True)
        sorted_group = shuffled_group.sort_values('end_date')
        return sorted_group

    final_data = final_data.groupby(['ean_global_channel', 'sub_axis'], group_keys=False).apply(shuffle_and_sort).reset_index(drop=True)
    final_data.drop(["start_date"], axis=1, inplace=True)
    final_data['seasonality_index'] = final_data['seasonality_index'].fillna(method='bfill')

    if fill_discontinuity:
        #  We Create a full date range for each ean_global_channel,
        full_data = []
        for name, group in final_data.groupby(['ean_global_channel']):
            group['end_date'] = pd.to_datetime(group['end_date'])
            group.set_index('end_date', inplace=True)
            full_range = pd.date_range(start= group.index.min(), end=group.index.max(), freq='W-SAT') #'10-08-2022'
            group = group.reindex(full_range).ffill().reset_index().rename(columns={'index': 'end_date'})
            full_data.append(group)
        final_data = pd.concat(full_data).reset_index(drop=True)

    result = final_data.groupby('ean_global_channel')['end_date'].agg(['min', 'max']).reset_index().sort_values(by='max', ascending=False)
    max_date_first_row = result.iloc[0]["max"]
    filtered_channels = result[result['max'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]
    final_data["end_date"] = pd.to_datetime(final_data["end_date"])
    final_data["year"] = final_data["end_date"].dt.year
    final_data["month"] = final_data["end_date"].dt.month
    final_data["week"] = final_data["end_date"].dt.isocalendar().week

    train_set = final_data.loc[((final_data['year'] == 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]


    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]

    # Filter the ean_global_channel in result where max date is less than the max date of the first row
    filtered_channels = ean_test_date[ean_test_date['end_date'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    # Filter the original DataFrame based on the filtered ean_global_channel
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] == 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]
    print("final data product (if changed we remove discontinuity)", final_data.ean_global_channel.unique().shape[0] )
    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]
    min_date_first_row = ean_test_date.iloc[0]["end_date"]
    print("prediction length:", max_date_first_row)
    assert min_date_first_row == max_date_first_row , "min_date_first_row != max_date_first_row"


    return train_set, test_set, max_date_first_row

In [266]:
def import_all(client, zero_percent, month,num_weeks, channel=None, fill_discontinuity=False, keep_non_promo=False, interpolation_method=1):
    def query(zero_percent, keep_non_promo = False):
        if keep_non_promo:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                """.format(zero_percent)
        else:
            a = """
            WITH MinPromoDate AS (
                SELECT
                    MIN(end_date) AS min_date
                FROM
                    `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                WHERE
                    is_promo = TRUE
            ),
            TransformedData AS (
                SELECT
                    start_date,
                    end_date,
                    sub_axis,
                    ean,
                    global_channel_type,
                    seasonality_index,
                    CASE
                        WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                        ELSE price_range
                    END AS price_range,
                    sold_units,
                    CASE
                        WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                        ELSE sub_tactic
                    END AS sub_tactic,
                    CASE
                        WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                        ELSE is_promo
                    END AS is_promo
                FROM
                    `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                WHERE
                    ean IS NOT NULL AND
                    end_date IS NOT NULL
            ),
            EANThreshold AS (
                SELECT
                    ean,
                    global_channel_type,
                    SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                FROM
                    TransformedData
                GROUP BY
                    ean,
                    global_channel_type
                HAVING
                    ZeroPercent <= {}
            ),
            PromoEANs AS (
                SELECT
                    ean,
                    global_channel_type,
                    SUM(CASE WHEN is_promo = TRUE THEN 1 ELSE 0 END) > 0 AS has_promo
                FROM
                    TransformedData
                GROUP BY
                    ean,
                    global_channel_type
                HAVING
                    has_promo
            )
            SELECT
                td.start_date,
                td.end_date,
                td.sub_axis,
                td.ean,
                td.global_channel_type,
                td.seasonality_index,
                td.price_range,
                td.is_promo,
                td.sub_tactic,
                td.sold_units
            FROM
                TransformedData td
            JOIN
                EANThreshold et
            ON
                td.ean = et.ean
                AND td.global_channel_type = et.global_channel_type
            JOIN
                PromoEANs pe
            ON
                td.ean = pe.ean
                AND td.global_channel_type = pe.global_channel_type
            """.format(zero_percent)

        if channel=='Online': 
            a+="""where td.global_channel_type = 'Online'
            ORDER BY
                td.end_date;"""
        elif channel=='Offline':
            a+="""where td.global_channel_type = 'Offline'
            ORDER BY
                td.end_date;"""
        else:
            a+="""
            ORDER BY
                td.end_date;"""
        return a

    data =client.query_and_wait(query(zero_percent, keep_non_promo)).to_dataframe()
    print("number of products before preprocessing", data["ean"].unique().shape[0])



    # Step 1: Count unique end dates for each EAN and each global channel type
    unique_dates = data.groupby(['ean', 'global_channel_type'])['end_date'].nunique().reset_index()

    # Step 2: Filter to find EANs where each global channel type has more than 65 unique dates
    eans_77_dates = unique_dates[unique_dates['end_date'] >= num_weeks]
    valid_eans = eans_77_dates.groupby('ean').filter(lambda x: len(x) == data["global_channel_type"].unique().shape[0] and all(x['end_date'] >= num_weeks))

    # Step 3: Filter the original DataFrame to include only these EANs
    data = data[data['ean'].isin(valid_eans['ean'])]
    data["sold_units"] = data["sold_units"].astype(float)
    data = data.sort_values(by=["end_date", "global_channel_type", "ean"])

    data['ean_global_channel'] = data['ean'] + '_' + data['global_channel_type']
    data['sub_tactic'] = data['sub_tactic'].str.lower().str.strip()

    def aggregate_subtactics(series):
        if series is None or all(pd.isnull(series)): 
            return ''
        all_subtactics = set()
        for items in series.dropna():
            tactics = set(item.strip() for item in items.split(','))
            all_subtactics.update(tactics)
        return ', '.join(sorted(all_subtactics))

    def custom_price_range(series):
        return series.mean(skipna=True) if not series.isnull().all() else np.nan

    aggregated_data = data.groupby(['start_date', 'end_date', 'ean_global_channel']).agg({
        'is_promo': 'first',
        'price_range': custom_price_range,
        'sub_tactic': aggregate_subtactics,
        'sub_axis': 'first',
        'seasonality_index': 'first',
        'sold_units': 'first'
    }).reset_index()

    aggregated_data.drop_duplicates(inplace=True)
    print("How many ean_global_channel_type:", aggregated_data.ean_global_channel.unique().shape[0])
    one_hot_encoded_data = aggregated_data['sub_tactic'].str.get_dummies(', ')
    empty_sub_tactic_indices = aggregated_data[aggregated_data['sub_tactic'] == ''].index
    one_hot_encoded_data.loc[empty_sub_tactic_indices] = 0

    final_data = pd.concat([aggregated_data, one_hot_encoded_data], axis=1)
    final_data.drop(['sub_tactic'], axis=1, inplace=True)

    def shuffle_and_sort(group):
        shuffled_group = group.sample(frac=1).reset_index(drop=True)
        sorted_group = shuffled_group.sort_values('end_date')
        return sorted_group

    final_data = final_data.groupby(['ean_global_channel', 'sub_axis'], group_keys=False).apply(shuffle_and_sort).reset_index(drop=True)
    final_data.drop(["start_date"], axis=1, inplace=True)
    final_data['seasonality_index'] = final_data['seasonality_index'].fillna(method='bfill')

    if fill_discontinuity:
        #  We Create a full date range for each ean_global_channel,
        full_data = []
        for name, group in final_data.groupby(['ean_global_channel']):
            group['end_date'] = pd.to_datetime(group['end_date'])
            group.set_index('end_date', inplace=True)
            full_range = pd.date_range(start= group.index.min(), end=group.index.max(), freq='W-SAT') #'10-08-2022'
            group = group.reindex(full_range).ffill().reset_index().rename(columns={'index': 'end_date'})
            full_data.append(group)
        final_data = pd.concat(full_data).reset_index(drop=True)

    result = final_data.groupby('ean_global_channel')['end_date'].agg(['min', 'max']).reset_index().sort_values(by='max', ascending=False)
    max_date_first_row = result.iloc[0]["max"]
    filtered_channels = result[result['max'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]
    final_data["end_date"] = pd.to_datetime(final_data["end_date"])
    final_data["year"] = final_data["end_date"].dt.year
    final_data["month"] = final_data["end_date"].dt.month
    final_data["week"] = final_data["end_date"].dt.isocalendar().week

    train_set = final_data.loc[((final_data['year'] <= 2023) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]


    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]

    # Filter the ean_global_channel in result where max date is less than the max date of the first row
    filtered_channels = ean_test_date[ean_test_date['end_date'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    # Filter the original DataFrame based on the filtered ean_global_channel
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] <= 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]
    print("final data product (if changed we remove discontinuity)", final_data.ean_global_channel.unique().shape[0] )
    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]
    min_date_first_row = ean_test_date.iloc[0]["end_date"]
    print("prediction length:", max_date_first_row)
    assert min_date_first_row == max_date_first_row , "min_date_first_row != max_date_first_row"

    ##################################################################################################
    #######################INTERPOLATION STEP#########################################################
    if interpolation_method==1:
        data=final_data.copy()
        data['is_promo'] = data['is_promo'].apply(lambda x: 1 if x is True else (0 if x is False else np.nan))
        # Encoding categorical variables
        data['sub_axis_encoded'] = LabelEncoder().fit_transform(data['sub_axis'])
        data['sold_units'] = pd.to_numeric(data['sold_units'], errors='coerce')

        # Separate the dataset into training and prediction sets
        train_df = data[data['is_promo'].notna()]
        predict_df = data[data['is_promo'].isna()]

        # Split the training data into features and labels
        X = train_df[['sub_axis_encoded', 'sold_units']]
        y = train_df['is_promo']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the model
        xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='auc', colsample_bytree=1.0, eta=0.1, max_depth=6, min_child_weight=5, subsample=1.0)
        xgb_model.fit(X_train, y_train)

        # Predict on the testing set
        y_pred = xgb_model.predict(X_test)

        # Print the classification report and ROC-AUC score
        print(classification_report(y_test, y_pred))
        print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))

        xgb_model.fit(X, y)
        # Predict on the unlabeled data
        X_predict = predict_df[['sub_axis_encoded', 'sold_units']]
        predict_df['is_promo'] = xgb_model.predict(X_predict)

        # Merge the predictions back into the original dataset
        data.update(predict_df)
        def update_subtactics_and_price_(df):
            binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
            
            # Save the original promo indices and price range for later use
            original_promo_indices = df[(df['is_promo'] == 1) & (~df['price_range'].isna())].index

            if not original_promo_indices.empty:
                price_range_promo_true = df.loc[original_promo_indices, 'price_range'].mean()

                # Find common values for binary columns using the original promo values
                common_values_df = df.loc[original_promo_indices, binary_columns]

                
                if not common_values_df.empty:
                    common_values = common_values_df.mode().iloc[0]
                else:
                    common_values = pd.Series(0, index=binary_columns)  # Default to 0 if empty
                
                common_values = common_values.fillna(0)  # Ensure no NaNs in common values

            
                # Update rows where is_promo is 1 and original is_promo was NaN
                promo_indices = df[(df['is_promo'] == 1) & (df['price_range'].isna())].index

                
                df.loc[promo_indices, 'price_range'] = price_range_promo_true
                for col in binary_columns:
                    df.loc[promo_indices, col] = common_values[col]
            
            # Set subtactics and price to zero where is_promo is 0
            non_promo_indices = df[df['is_promo'] == 0].index

            
            df.loc[non_promo_indices, binary_columns] = 0
            df.loc[non_promo_indices, 'price_range'] = 0

            if original_promo_indices.empty:
                print(df.ean_global_channel.iloc[0])
            return df
        # Apply the function to update subtactics and price_range based on the new predictions
        result = data.groupby('ean_global_channel').apply(update_subtactics_and_price_).reset_index(drop=True)
        result = result.drop(["sub_axis_encoded"], axis=1)
    else :
        data = final_data.copy()
        result = data.groupby('ean_global_channel').apply(process_group).reset_index(drop=True)
        result['is_promo'] = result['predicted_promo']
        result = result.drop(["predicted_promo"], axis=1)
    
    ##################################################################################################
    #######################SPLITTING##################################################################
    final_data = result.copy()
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] <= 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]    

    assert max_date_first_row* 3 <num_weeks, "num weeks should be higher than 3 times prediction length"
    return final_data, train_set, test_set, max_date_first_row

In [7]:
train_set1, test_set1, prediction_len = import_true_promo(bq_client, 10, 12, 50,channel='Offline', fill_discontinuity=False, keep_non_promo=True)

number of products before preprocessing 2412
How many ean_global_channel_type: 1928
final data product (if changed we remove discontinuity) 1873
prediction length: 12


In [8]:

train_set1, test_set1, prediction_len = import_true_promo(bq_client, 10, 12, 50,channel='Offline', fill_discontinuity=True, keep_non_promo=False)

number of products before preprocessing 2083
How many ean_global_channel_type: 1801
final data product (if changed we remove discontinuity) 1784
prediction length: 12


In [267]:
final_data, train_set, test_set, prediction_len = import_all(bq_client, 10, 10, 100, channel= 'Offline', fill_discontinuity=False,
                                                            keep_non_promo=False, interpolation_method=1 )

number of products before preprocessing 2083
How many ean_global_channel_type: 1535
final data product (if changed we remove discontinuity) 1429
prediction length: 21
              precision    recall  f1-score   support

         0.0       0.61      0.12      0.19      2596
         1.0       0.89      0.99      0.94     19244

    accuracy                           0.89     21840
   macro avg       0.75      0.55      0.57     21840
weighted avg       0.86      0.89      0.85     21840

ROC-AUC Score: 0.5529852379898531


In [271]:
final_data, train_set, test_set, prediction_len = import_all(bq_client, 10, 10, 100, channel= 'Offline', fill_discontinuity=True,
                                                            keep_non_promo=False, interpolation_method=2 )

number of products before preprocessing 2083
How many ean_global_channel_type: 1535
final data product (if changed we remove discontinuity) 1508
prediction length: 21


In [15]:
train_set.isna().sum()

end_date                   0
ean_global_channel         0
is_promo              179754
price_range           179754
sub_axis                   0
seasonality_index          0
sold_units                 0
2 for a price              0
3 for 2                    0
bogof                      0
bogshp                     0
coupon                     0
listing fee                0
online                     0
save                       0
site fee                   0
year                       0
month                      0
week                       0
dtype: int64

### Interpolation:
For this we only intrested in a function that takes a ean_global_channel and apply the following changes:
- interpolate all positive is_promo and interpolate corresponding subtactics and price_rand
- for negative is_promo we change price_range by zero

In [16]:
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [260]:
data = final_data.copy()

In [261]:
data.isna().sum()

end_date                   0
ean_global_channel         0
is_promo              179780
price_range           179780
sub_axis                   0
seasonality_index          0
sold_units                 0
2 for a price              0
3 for 2                    0
bogof                      0
bogshp                     0
coupon                     0
listing fee                0
online                     0
save                       0
site fee                   0
year                       0
month                      0
week                       0
dtype: int64

In [262]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# Function to impute missing values
def impute_missing_values(df):
    df['is_promo'] = df['is_promo'].apply(lambda x: 1 if x is True else (-1 if x is False else np.nan))
    return df

# Function to prepare data for PU
def prepare_data(df):
    df = impute_missing_values(df)
    dff = df[['price_range', 'sold_units', '2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee', 'is_promo']].copy()
    pos_ind = np.where(dff['is_promo'] == 1)[0]
    if len(pos_ind) == 0:
        return None, None, None
    np.random.shuffle(pos_ind)
    pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    dff.reset_index(drop=True, inplace=True)
    dff['class_test'] = -1
    dff.loc[pos_sample, 'class_test'] = 1

    X_data = dff['sold_units'].values.reshape(-1, 1)  # Reshape to 2D array for XGBoost
    y_labeled = dff['class_test'].values
    y_positive = dff['is_promo'].values
    return X_data, y_labeled, y_positive

# Function to fit PU estimator
def fit_PU_estimator(X, y, hold_out_ratio, estimator):
    positives = np.where(y == 1.0)[0]
    hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
    if hold_out_size == 0:
        return estimator, 1.0  # Handle case where there are no hold-out samples
    np.random.shuffle(positives)
    hold_out = positives[:hold_out_size]
    X_hold_out = X[hold_out]
    X = np.delete(X, hold_out, 0)
    y = np.delete(y, hold_out)
    
    estimator.fit(X, y)
    hold_out_predictions = estimator.predict_proba(X_hold_out)[:, 1]
    c = np.mean(hold_out_predictions)
    return estimator, c

# Function to predict PU probabilities
def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)[:, 1]
    return predicted_s / prob_s1y1

# Function to perform positive unlabeling
def positive_unlabeling(df):
    X_data, y_labeled, y_positive = prepare_data(df)
    if X_data is None or y_labeled is None:
        df['predicted_promo'] = df['is_promo']
        return df
    y_labeled[y_labeled == -1] = 0
    predicted = np.zeros(len(X_data))
    learning_iterations = 24

    for index in range(learning_iterations):
        pu_estimator, probs1y1 = fit_PU_estimator(X_data, y_labeled, 0.2, XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
        predicted += predict_PU_prob(X_data, pu_estimator, probs1y1)
    
    y_predict = [1 if x > 0.9 else 0 for x in (predicted / learning_iterations)]
    df['predicted_promo'] = y_predict
    return df

# Function to update subtactics and price
def update_subtactics_and_price(df):
    binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
    
    # Save the true promo indices where both predicted_promo and is_promo are 1
    true_promo_indices = df[(df['predicted_promo'] == 1) & (df['is_promo'] == 1)].index

    if not true_promo_indices.empty:
        # Compute mean price range for true promo values
        price_range_promo_true = df.loc[true_promo_indices, 'price_range'].mean()

        # Find common values for binary columns using true promo values
        common_values_df = df.loc[true_promo_indices, binary_columns]
        
        if not common_values_df.empty:
            common_values = common_values_df.mode().iloc[0]
        else:
            common_values = pd.Series(0, index=binary_columns)  # Default to 0 if empty
        
        # Ensure no NaNs in common values
        common_values = common_values.fillna(0)

        # Update rows where predicted_promo is 1 and original is_promo was NaN
        promo_indices = df[(df['predicted_promo'] == 1) & (df['is_promo'].isna())].index
        df.loc[promo_indices, 'price_range'] = price_range_promo_true
        for col in binary_columns:
            df.loc[promo_indices, col] = common_values[col]
    
    # Set subtactics and price to zero where predicted_promo is 0
    non_promo_indices = df[df['predicted_promo'] == 0].index
    df.loc[non_promo_indices, binary_columns] = 0
    df.loc[non_promo_indices, 'price_range'] = 0

    return df

# Apply the process to each ean_global_channel group
def process_group(group):
    group = positive_unlabeling(group)
    group = update_subtactics_and_price(group)
    return group


In [263]:
# Apply the processing function to each ean_global_channel group
result = data.groupby('ean_global_channel').apply(process_group).reset_index(drop=True)

In [264]:
result.isna().sum()

end_date                   0
ean_global_channel         0
is_promo              179780
price_range                0
sub_axis                   0
seasonality_index          0
sold_units                 0
2 for a price              0
3 for 2                    0
bogof                      0
bogshp                     0
coupon                     0
listing fee                0
online                     0
save                       0
site fee                   0
year                       0
month                      0
week                       0
predicted_promo            0
dtype: int64

In [239]:
data=final_data.copy()

In [154]:
data.isna().sum()

end_date                   0
ean_global_channel         0
is_promo              179780
price_range           179780
sub_axis                   0
seasonality_index          0
sold_units                 0
2 for a price              0
3 for 2                    0
bogof                      0
bogshp                     0
coupon                     0
listing fee                0
online                     0
save                       0
site fee                   0
year                       0
month                      0
week                       0
dtype: int64

In [155]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

data['is_promo'] = data['is_promo'].apply(lambda x: 1 if x is True else (0 if x is False else np.nan))
# Encoding categorical variables
data['sub_axis_encoded'] = LabelEncoder().fit_transform(data['sub_axis'])
data['sold_units'] = pd.to_numeric(data['sold_units'], errors='coerce')

# Separate the dataset into training and prediction sets
train_df = data[data['is_promo'].notna()]
predict_df = data[data['is_promo'].isna()]

# Split the training data into features and labels
X = train_df[['sub_axis_encoded', 'sold_units']]
y = train_df['is_promo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='auc', colsample_bytree=1.0, eta=0.1, max_depth=6, min_child_weight=5, subsample=1.0)
xgb_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = xgb_model.predict(X_test)

# Print the classification report and ROC-AUC score
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))

xgb_model.fit(X, y)
# Predict on the unlabeled data
X_predict = predict_df[['sub_axis_encoded', 'sold_units']]
predict_df['is_promo'] = xgb_model.predict(X_predict)

# Merge the predictions back into the original dataset
data.update(predict_df)



              precision    recall  f1-score   support

         0.0       0.63      0.15      0.24      2847
         1.0       0.89      0.99      0.94     20212

    accuracy                           0.88     23059
   macro avg       0.76      0.57      0.59     23059
weighted avg       0.86      0.88      0.85     23059

ROC-AUC Score: 0.5667982435012194


In [111]:
df = data.copy()
binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
original_promo_indices = df[(df['is_promo'] == 1) & (~df['price_range'].isna())].index
price_range_promo_true = df.loc[original_promo_indices, 'price_range'].mean()
common_values = df.loc[original_promo_indices, binary_columns].mode().iloc[0]
common_values

2 for a price    0.0
3 for 2          0.0
bogof            0.0
bogshp           0.0
coupon           0.0
listing fee      0.0
online           1.0
save             1.0
site fee         1.0
Name: 0, dtype: float64

In [156]:
def update_subtactics_and_price(df):
    binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
    
    # Save the original promo indices and price range for later use
    original_promo_indices = df[(df['is_promo'] == 1) & (~df['price_range'].isna())].index

    if not original_promo_indices.empty:
        price_range_promo_true = df.loc[original_promo_indices, 'price_range'].mean()

        # Find common values for binary columns using the original promo values
        common_values_df = df.loc[original_promo_indices, binary_columns]

        
        if not common_values_df.empty:
            common_values = common_values_df.mode().iloc[0]
        else:
            common_values = pd.Series(0, index=binary_columns)  # Default to 0 if empty
        
        common_values = common_values.fillna(0)  # Ensure no NaNs in common values

    
        # Update rows where is_promo is 1 and original is_promo was NaN
        promo_indices = df[(df['is_promo'] == 1) & (df['price_range'].isna())].index

        
        df.loc[promo_indices, 'price_range'] = price_range_promo_true
        for col in binary_columns:
            df.loc[promo_indices, col] = common_values[col]
    
    # Set subtactics and price to zero where is_promo is 0
    non_promo_indices = df[df['is_promo'] == 0].index

    
    df.loc[non_promo_indices, binary_columns] = 0
    df.loc[non_promo_indices, 'price_range'] = 0

    if original_promo_indices.empty:
        print(df.ean_global_channel.iloc[0])
    return df
# Apply the function to update subtactics and price_range based on the new predictions
result_class = data.groupby('ean_global_channel').apply(update_subtactics_and_price).reset_index(drop=True)


In [243]:
result_class[result_class.ean_global_channel=='800897848934_Offline'].groupby('is_promo').count()

,end_date,ean_global_channel,price_range,sub_axis,seasonality_index,sold_units,2 for a price,3 for 2,bogof,bogshp,coupon,listing fee,online,save,site fee,year,month,week,sub_axis_encoded
is_promo,,,,,,,,,,,,,,,,,,,
0.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
1.0,208,208,208,208,208,208,208,208,208,208,208,208,208,208,208,208,208,208,208


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import os
import pickle


####################################################################################################################
###########################################INTERPOLATION METHODS####################################################

# Function to prepare data for PU
def prepare_data(df):
    df['is_promo'] = df['is_promo'].apply(lambda x: 1 if x is True else (-1 if x is False else np.nan))
    dff = df[['price_range', 'sold_units', '2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee', 'is_promo']].copy()
    pos_ind = np.where(dff['is_promo'] == 1)[0]
    if len(pos_ind) == 0:
        return None, None, None
    np.random.shuffle(pos_ind)
    pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    dff.reset_index(drop=True, inplace=True)
    dff['class_test'] = -1
    dff.loc[pos_sample, 'class_test'] = 1

    X_data = dff['sold_units'].values.reshape(-1, 1)  # Reshape to 2D array for XGBoost
    y_labeled = dff['class_test'].values
    y_positive = dff['is_promo'].values
    return X_data, y_labeled, y_positive

# Function to fit PU estimator
def fit_PU_estimator(X, y, hold_out_ratio, estimator):
    positives = np.where(y == 1.0)[0]
    hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
    if hold_out_size == 0:
        return estimator, 1.0  # Handle case where there are no hold-out samples
    np.random.shuffle(positives)
    hold_out = positives[:hold_out_size]
    X_hold_out = X[hold_out]
    X = np.delete(X, hold_out, 0)
    y = np.delete(y, hold_out)
    
    estimator.fit(X, y)
    hold_out_predictions = estimator.predict_proba(X_hold_out)[:, 1]
    c = np.mean(hold_out_predictions)
    return estimator, c

# Function to predict PU probabilities
def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)[:, 1]
    return predicted_s / prob_s1y1

# Function to perform positive unlabeling
def positive_unlabeling(df):
    X_data, y_labeled, y_positive = prepare_data(df)
    if X_data is None or y_labeled is None:
        df['predicted_promo'] = df['is_promo']
        return df
    y_labeled[y_labeled == -1] = 0
    predicted = np.zeros(len(X_data))
    learning_iterations = 24

    for index in range(learning_iterations):
        pu_estimator, probs1y1 = fit_PU_estimator(X_data, y_labeled, 0.2, XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
        predicted += predict_PU_prob(X_data, pu_estimator, probs1y1)
    
    y_predict = [1 if x > 0.9 else 0 for x in (predicted / learning_iterations)]
    df['predicted_promo'] = y_predict
    return df

# Function to update subtactics and price
def update_subtactics_and_price(df):
    binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
    
    # Save the true promo indices where both predicted_promo and is_promo are 1
    true_promo_indices = df[(df['predicted_promo'] == 1) & (df['is_promo'] == 1)].index

    if not true_promo_indices.empty:
        # Compute mean price range for true promo values
        price_range_promo_true = df.loc[true_promo_indices, 'price_range'].mean()

        # Find common values for binary columns using true promo values
        common_values_df = df.loc[true_promo_indices, binary_columns]
        
        if not common_values_df.empty:
            common_values = common_values_df.mode().iloc[0]
        else:
            common_values = pd.Series(0, index=binary_columns)  # Default to 0 if empty
        
        # Ensure no NaNs in common values
        common_values = common_values.fillna(0)

        # Update rows where predicted_promo is 1 and original is_promo was NaN
        promo_indices = df[(df['predicted_promo'] == 1) & (df['is_promo'].isna())].index
        df.loc[promo_indices, 'price_range'] = price_range_promo_true
        for col in binary_columns:
            df.loc[promo_indices, col] = common_values[col]
    
    # Set subtactics and price to zero where predicted_promo is 0
    non_promo_indices = df[df['predicted_promo'] == 0].index
    df.loc[non_promo_indices, binary_columns] = 0
    df.loc[non_promo_indices, 'price_range'] = 0

    return df

# Apply the process to each ean_global_channel group
def process_group(group):
    group = positive_unlabeling(group)
    group = update_subtactics_and_price(group)
    return group


def import_true_promo(client, zero_percent, month, num_weeks,channel=None, fill_discontinuity=False, keep_non_promo=False):

    """
    This function download data From gcp
    Options:
    - zero_percentage os sales values == 0
    - month to do the split train/test
    - num_weeks minimal we'll add assert num_weeks>3*prediction_length
    - channel Both, offline, online
    -fill_discontinuity: add the product that has discounuity in values and interpolate them
    - keep non_propo: true means we also keep the product that has no promotions during the whole period
    """
    def query(zero_percent, keep_non_promo = False):
        

        if keep_non_promo:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                WHERE
                    td.end_date >= (SELECT min_date FROM MinPromoDate)
                """.format(zero_percent)
        else:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                ),
                PromoFilter AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN is_promo = TRUE THEN 1 ELSE 0 END) > 0 AS has_promo
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        has_promo
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                JOIN
                    PromoFilter pf
                ON
                    td.ean = pf.ean
                    AND td.global_channel_type = pf.global_channel_type
                WHERE
                    td.end_date >= (SELECT min_date FROM MinPromoDate)
                """.format(zero_percent)
        if channel=='Online': 
            a+="""AND td.global_channel_type = 'Online'
            ORDER BY
                td.end_date;"""
        elif channel=='Offline':
            a+="""AND td.global_channel_type = 'Offline'
            ORDER BY
                td.end_date;"""
        else:
            a+="""
            ORDER BY
                td.end_date;"""
        return a

    data['ean_global_channel'] = data['ean'] + '_' + data['global_channel_type']
    print("number of products before preprocessing", data["ean_global_channel"].unique().shape[0])


    # Step 1: Count unique end dates for each ean_global_channel
    unique_dates = data.groupby('ean_global_channel')['end_date'].nunique().reset_index()

    # Step 2: Filter to find ean_global_channels with more than or equal to num_weeks unique dates
    valid_ean_global_channels = unique_dates[unique_dates['end_date'] >= num_weeks]['ean_global_channel']

    # Step 3: Filter the original DataFrame to include only these ean_global_channels
    data = data[data['ean_global_channel'].isin(valid_ean_global_channels)]

    data['sub_tactic'] = data['sub_tactic'].str.lower().str.strip()

    def aggregate_subtactics(series):
        if series is None or all(pd.isnull(series)): 
            return ''
        all_subtactics = set()
        for items in series.dropna():
            tactics = set(item.strip() for item in items.split(','))
            all_subtactics.update(tactics)
        return ', '.join(sorted(all_subtactics))

    def custom_price_range(series):
        return series.mean(skipna=True) if not series.isnull().all() else np.nan

    aggregated_data = data.groupby(['start_date', 'end_date', 'ean_global_channel']).agg({
        'is_promo': 'first',
        'price_range': custom_price_range,
        'sub_tactic': aggregate_subtactics,
        'sub_axis': 'first',
        'seasonality_index': 'first',
        'sold_units': 'first'
    }).reset_index()

    aggregated_data.drop_duplicates(inplace=True)
    print("How many ean_global_channel_type:", aggregated_data.ean_global_channel.unique().shape[0])
    if aggregated_data.ean_global_channel.unique().shape[0] == 0:
        raise ValueError("Error: No unique ean_global_channel values found.")
    one_hot_encoded_data = aggregated_data['sub_tactic'].str.get_dummies(', ')
    empty_sub_tactic_indices = aggregated_data[aggregated_data['sub_tactic'] == ''].index
    one_hot_encoded_data.loc[empty_sub_tactic_indices] = 0

    final_data = pd.concat([aggregated_data, one_hot_encoded_data], axis=1)
    final_data.drop(['sub_tactic'], axis=1, inplace=True)

    def shuffle_and_sort(group):
        shuffled_group = group.sample(frac=1).reset_index(drop=True)
        sorted_group = shuffled_group.sort_values('end_date')
        return sorted_group

    final_data = final_data.groupby(['ean_global_channel', 'sub_axis'], group_keys=False).apply(shuffle_and_sort).reset_index(drop=True)
    final_data.drop(["start_date"], axis=1, inplace=True)
    final_data['seasonality_index'] = final_data['seasonality_index'].fillna(method='bfill')

    if fill_discontinuity:
        #  We Create a full date range for each ean_global_channel,
        full_data = []
        for name, group in final_data.groupby(['ean_global_channel']):
            group['end_date'] = pd.to_datetime(group['end_date'])
            group.set_index('end_date', inplace=True)
            full_range = pd.date_range(start= group.index.min(), end=group.index.max(), freq='W-SAT') #'10-08-2022'
            group = group.reindex(full_range).ffill().reset_index().rename(columns={'index': 'end_date'})
            full_data.append(group)
        final_data = pd.concat(full_data).reset_index(drop=True)

    result = final_data.groupby('ean_global_channel')['end_date'].agg(['min', 'max']).reset_index().sort_values(by='max', ascending=False)
    max_date_first_row = result.iloc[0]["max"]
    filtered_channels = result[result['max'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]
    final_data["end_date"] = pd.to_datetime(final_data["end_date"])
    final_data["year"] = final_data["end_date"].dt.year
    final_data["month"] = final_data["end_date"].dt.month
    final_data["week"] = final_data["end_date"].dt.isocalendar().week

    train_set = final_data.loc[((final_data['year'] == 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]


    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]

    # Filter the ean_global_channel in result where max date is less than the max date of the first row
    filtered_channels = ean_test_date[ean_test_date['end_date'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    # Filter the original DataFrame based on the filtered ean_global_channel
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] == 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]
    print("final data product (if changed we remove discontinuity)", final_data.ean_global_channel.unique().shape[0] )
    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]
    min_date_first_row = ean_test_date.iloc[0]["end_date"]
    print("prediction length:", max_date_first_row)
    assert min_date_first_row == max_date_first_row , "min_date_first_row != max_date_first_row"


    return final_data, train_set, test_set, max_date_first_row


def import_all(client, zero_percent, month,num_weeks, channel=None, fill_discontinuity=False, keep_non_promo=False, interpolation_method=True):
    def query(zero_percent, keep_non_promo = False):
        if keep_non_promo:
            a = """
                WITH MinPromoDate AS (
                    SELECT
                        MIN(end_date) AS min_date
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        is_promo = TRUE
                ),
                TransformedData AS (
                    SELECT
                        start_date,
                        end_date,
                        sub_axis,
                        ean,
                        global_channel_type,
                        seasonality_index,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                            ELSE price_range
                        END AS price_range,
                        sold_units,
                        CASE
                            WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                            ELSE sub_tactic
                        END AS sub_tactic,
                        CASE
                            WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                            ELSE is_promo
                        END AS is_promo
                    FROM
                        `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                    WHERE
                        ean IS NOT NULL AND
                        end_date IS NOT NULL
                ),
                EANThreshold AS (
                    SELECT
                        ean,
                        global_channel_type,
                        SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                    FROM
                        TransformedData
                    GROUP BY
                        ean,
                        global_channel_type
                    HAVING
                        ZeroPercent <= {}
                )
                SELECT
                    td.start_date,
                    td.end_date,
                    td.sub_axis,
                    td.ean,
                    td.global_channel_type,
                    td.seasonality_index,
                    td.price_range,
                    td.is_promo,
                    td.sub_tactic,
                    td.sold_units
                FROM
                    TransformedData td
                JOIN
                    EANThreshold et
                ON
                    td.ean = et.ean
                    AND td.global_channel_type = et.global_channel_type
                """.format(zero_percent)
        else:
            a = """
            WITH MinPromoDate AS (
                SELECT
                    MIN(end_date) AS min_date
                FROM
                    `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                WHERE
                    is_promo = TRUE
            ),
            TransformedData AS (
                SELECT
                    start_date,
                    end_date,
                    sub_axis,
                    ean,
                    global_channel_type,
                    seasonality_index,
                    CASE
                        WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN 0
                        ELSE price_range
                    END AS price_range,
                    sold_units,
                    CASE
                        WHEN is_promo = FALSE AND end_date >= (SELECT min_date FROM MinPromoDate) THEN ''
                        ELSE sub_tactic
                    END AS sub_tactic,
                    CASE
                        WHEN is_promo = FALSE AND end_date < (SELECT min_date FROM MinPromoDate) THEN NULL
                        ELSE is_promo
                    END AS is_promo
                FROM
                    `itg-bpma-gbl-ww-np.bpma_ds_c2_exposed_eu_np.pnl_details_sellout_no_fakes`
                WHERE
                    ean IS NOT NULL AND
                    end_date IS NOT NULL
            ),
            EANThreshold AS (
                SELECT
                    ean,
                    global_channel_type,
                    SUM(CASE WHEN sold_units = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS ZeroPercent
                FROM
                    TransformedData
                GROUP BY
                    ean,
                    global_channel_type
                HAVING
                    ZeroPercent <= {}
            ),
            PromoEANs AS (
                SELECT
                    ean,
                    global_channel_type,
                    SUM(CASE WHEN is_promo = TRUE THEN 1 ELSE 0 END) > 0 AS has_promo
                FROM
                    TransformedData
                GROUP BY
                    ean,
                    global_channel_type
                HAVING
                    has_promo
            )
            SELECT
                td.start_date,
                td.end_date,
                td.sub_axis,
                td.ean,
                td.global_channel_type,
                td.seasonality_index,
                td.price_range,
                td.is_promo,
                td.sub_tactic,
                td.sold_units
            FROM
                TransformedData td
            JOIN
                EANThreshold et
            ON
                td.ean = et.ean
                AND td.global_channel_type = et.global_channel_type
            JOIN
                PromoEANs pe
            ON
                td.ean = pe.ean
                AND td.global_channel_type = pe.global_channel_type
            """.format(zero_percent)

        if channel=='Online': 
            a+="""where td.global_channel_type = 'Online'
            ORDER BY
                td.end_date;"""
        elif channel=='Offline':
            a+="""where td.global_channel_type = 'Offline'
            ORDER BY
                td.end_date;"""
        else:
            a+="""
            ORDER BY
                td.end_date;"""
        return a

    data =client.query_and_wait(query(zero_percent, keep_non_promo)).to_dataframe()
    data['ean_global_channel'] = data['ean'] + '_' + data['global_channel_type']
    print("number of products before preprocessing", data["ean_global_channel"].unique().shape[0])



    

    # Step 1: Count unique end dates for each ean_global_channel
    unique_dates = data.groupby('ean_global_channel')['end_date'].nunique().reset_index()

    # Step 2: Filter to find ean_global_channels with more than or equal to num_weeks unique dates
    valid_ean_global_channels = unique_dates[unique_dates['end_date'] >= num_weeks]['ean_global_channel']

    # Step 3: Filter the original DataFrame to include only these ean_global_channels
    data = data[data['ean_global_channel'].isin(valid_ean_global_channels)]

    # Convert 'sold_units' to float
    data["sold_units"] = data["sold_units"].astype(float)

    # Sort the data
    data = data.sort_values(by=["end_date", "global_channel_type", "ean"])
    data['sub_tactic'] = data['sub_tactic'].str.lower().str.strip()

    def aggregate_subtactics(series):
        if series is None or all(pd.isnull(series)): 
            return ''
        all_subtactics = set()
        for items in series.dropna():
            tactics = set(item.strip() for item in items.split(','))
            all_subtactics.update(tactics)
        return ', '.join(sorted(all_subtactics))

    def custom_price_range(series):
        return series.mean(skipna=True) if not series.isnull().all() else np.nan

    aggregated_data = data.groupby(['start_date', 'end_date', 'ean_global_channel']).agg({
        'is_promo': 'first',
        'price_range': custom_price_range,
        'sub_tactic': aggregate_subtactics,
        'sub_axis': 'first',
        'seasonality_index': 'first',
        'sold_units': 'first'
    }).reset_index()

    aggregated_data.drop_duplicates(inplace=True)
    print("How many ean_global_channel_type:", aggregated_data.ean_global_channel.unique().shape[0])
    if aggregated_data.ean_global_channel.unique().shape[0] == 0:
        raise ValueError("Error: No unique ean_global_channel values found.")
    one_hot_encoded_data = aggregated_data['sub_tactic'].str.get_dummies(', ')
    empty_sub_tactic_indices = aggregated_data[aggregated_data['sub_tactic'] == ''].index
    one_hot_encoded_data.loc[empty_sub_tactic_indices] = 0

    final_data = pd.concat([aggregated_data, one_hot_encoded_data], axis=1)
    final_data.drop(['sub_tactic'], axis=1, inplace=True)

    def shuffle_and_sort(group):
        shuffled_group = group.sample(frac=1).reset_index(drop=True)
        sorted_group = shuffled_group.sort_values('end_date')
        return sorted_group

    final_data = final_data.groupby(['ean_global_channel', 'sub_axis'], group_keys=False).apply(shuffle_and_sort).reset_index(drop=True)
    final_data.drop(["start_date"], axis=1, inplace=True)
    final_data['seasonality_index'] = final_data['seasonality_index'].fillna(method='bfill')

    if fill_discontinuity:
        #  We Create a full date range for each ean_global_channel,
        full_data = []
        for name, group in final_data.groupby(['ean_global_channel']):
            group['end_date'] = pd.to_datetime(group['end_date'])
            group.set_index('end_date', inplace=True)
            full_range = pd.date_range(start= group.index.min(), end=group.index.max(), freq='W-SAT') #'10-08-2022'
            group = group.reindex(full_range).ffill().reset_index().rename(columns={'index': 'end_date'})
            full_data.append(group)
        final_data = pd.concat(full_data).reset_index(drop=True)

    result = final_data.groupby('ean_global_channel')['end_date'].agg(['min', 'max']).reset_index().sort_values(by='max', ascending=False)
    max_date_first_row = result.iloc[0]["max"]
    filtered_channels = result[result['max'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]
    final_data["end_date"] = pd.to_datetime(final_data["end_date"])
    final_data["year"] = final_data["end_date"].dt.year
    final_data["month"] = final_data["end_date"].dt.month
    final_data["week"] = final_data["end_date"].dt.isocalendar().week

    train_set = final_data.loc[((final_data['year'] <= 2023) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]


    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]

    # Filter the ean_global_channel in result where max date is less than the max date of the first row
    filtered_channels = ean_test_date[ean_test_date['end_date'] < max_date_first_row]['ean_global_channel'].reset_index(drop=True)

    # Filter the original DataFrame based on the filtered ean_global_channel
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] <= 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]
    print("final data product (if changed we remove discontinuity)", final_data.ean_global_channel.unique().shape[0] )
    ean_test_date = test_set.groupby("ean_global_channel").end_date.count().reset_index().sort_values('end_date')
    max_date_first_row = ean_test_date.iloc[-1]["end_date"]
    min_date_first_row = ean_test_date.iloc[0]["end_date"]
    print("prediction length:", max_date_first_row)
    assert min_date_first_row == max_date_first_row , "min_date_first_row != max_date_first_row"

    ##################################################################################################
    #######################INTERPOLATION STEP#########################################################
    print("Interpolation step starting now")
    if interpolation_method==False:
        data=final_data.copy()
        data['is_promo'] = data['is_promo'].apply(lambda x: 1 if x is True else (0 if x is False else np.nan))
        # Encoding categorical variables
        data['sub_axis_encoded'] = LabelEncoder().fit_transform(data['sub_axis'])
        data['sold_units'] = pd.to_numeric(data['sold_units'], errors='coerce')

        # Separate the dataset into training and prediction sets
        train_df = data[data['is_promo'].notna()]
        predict_df = data[data['is_promo'].isna()]

        # Split the training data into features and labels
        X = train_df[['sub_axis_encoded', 'sold_units']]
        y = train_df['is_promo']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the model
        xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='auc', colsample_bytree=1.0, eta=0.1, max_depth=6, min_child_weight=5, subsample=1.0)
        xgb_model.fit(X_train, y_train)

        # Predict on the testing set
        y_pred = xgb_model.predict(X_test)

        # Print the classification report and ROC-AUC score
        print(classification_report(y_test, y_pred))
        print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))

        xgb_model.fit(X, y)
        # Predict on the unlabeled data
        X_predict = predict_df[['sub_axis_encoded', 'sold_units']]
        predict_df['is_promo'] = xgb_model.predict(X_predict)

        # Merge the predictions back into the original dataset
        data.update(predict_df)
        def update_subtactics_and_price_(df):
            binary_columns = ['2 for a price', '3 for 2', 'bogof', 'bogshp', 'coupon', 'listing fee', 'online', 'save', 'site fee']
            
            # Save the original promo indices and price range for later use
            original_promo_indices = df[(df['is_promo'] == 1) & (~df['price_range'].isna())].index

            if not original_promo_indices.empty:
                price_range_promo_true = df.loc[original_promo_indices, 'price_range'].mean()

                # Find common values for binary columns using the original promo values
                common_values_df = df.loc[original_promo_indices, binary_columns]

                
                if not common_values_df.empty:
                    common_values = common_values_df.mode().iloc[0]
                else:
                    common_values = pd.Series(0, index=binary_columns)  # Default to 0 if empty
                
                common_values = common_values.fillna(0)  # Ensure no NaNs in common values

            
                # Update rows where is_promo is 1 and original is_promo was NaN
                promo_indices = df[(df['is_promo'] == 1) & (df['price_range'].isna())].index

                
                df.loc[promo_indices, 'price_range'] = price_range_promo_true
                for col in binary_columns:
                    df.loc[promo_indices, col] = common_values[col]
            
            # Set subtactics and price to zero where is_promo is 0
            non_promo_indices = df[df['is_promo'] == 0].index

            
            df.loc[non_promo_indices, binary_columns] = 0
            df.loc[non_promo_indices, 'price_range'] = 0

            if original_promo_indices.empty:
                print(df.ean_global_channel.iloc[0])
            return df
        # Apply the function to update subtactics and price_range based on the new predictions
        result = data.groupby('ean_global_channel').apply(update_subtactics_and_price_).reset_index(drop=True)
        result = result.drop(["sub_axis_encoded"], axis=1)
    else :
        data = final_data.copy()
        result = data.groupby('ean_global_channel').apply(process_group).reset_index(drop=True)
        result['is_promo'] = result['predicted_promo']
        result = result.drop(["predicted_promo"], axis=1)
    
    print("Interpolation step is done")
    ##################################################################################################
    #######################SPLITTING##################################################################
    final_data = result.copy()
    final_data = final_data[~final_data['ean_global_channel'].isin(filtered_channels)]

    train_set = final_data.loc[((final_data['year'] <= 2022) | ((final_data['year'] == 2023) & (final_data['month'] <= month)))]
    test_set = final_data.loc[((final_data['year'] == 2023) & (final_data['month'] > month)) | (final_data['year'] == 2024)]    

    assert max_date_first_row* 3 <num_weeks, "num weeks should be higher than 3 times prediction length"
    return final_data, train_set, test_set, max_date_first_row


In [3]:
PROJECT_ID = "itg-bpma-gbl-ww-np"  # @param {type:"string"}
REGION = "europe-west1" 
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}
import vertexai
from google.cloud import bigquery
REMOTE_JOB_NAME = "timeseriesllm"
REMOTE_JOB_BUCKET = f"{BUCKET_URI}/{REMOTE_JOB_NAME}"
##################################################################################################
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=REMOTE_JOB_BUCKET,
)

##################################################################################################
bq_client = bigquery.Client(
    project=PROJECT_ID,  # GCP project used for running the queries and billing
)

In [7]:
final_data, train_set, test_set, prediction_len = import_all(bq_client, 0, 5, 172, channel= 'Offline', fill_discontinuity=False,
                                                            keep_non_promo=False, interpolation_method=False )

/home/user/miniconda/envs/vertex/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


number of products before preprocessing 1510
How many ean_global_channel_type: 858
final data product (if changed we remove discontinuity) 824
prediction length: 43
Interpolation step starting now
              precision    recall  f1-score   support

         0.0       0.70      0.16      0.26      1421
         1.0       0.90      0.99      0.95     11253

    accuracy                           0.90     12674
   macro avg       0.80      0.58      0.60     12674
weighted avg       0.88      0.90      0.87     12674

ROC-AUC Score: 0.5762670966216031


/tmp/ipykernel_40662/1837066132.py:720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_df['is_promo'] = xgb_model.predict(X_predict)
/tmp/ipykernel_40662/1837066132.py:764: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result = data.groupby('ean_global_channel').apply(update_subtactics_and_price_).reset_index(drop=True)


Interpolation step is done


In [8]:
final_data, train_set, test_set, prediction_len = import_all(bq_client, 0, 5, 172, channel= None, fill_discontinuity=False,
                                                            keep_non_promo=False, interpolation_method=False)

/home/user/miniconda/envs/vertex/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


number of products before preprocessing 2005
How many ean_global_channel_type: 1121
final data product (if changed we remove discontinuity) 1071
prediction length: 43
Interpolation step starting now
              precision    recall  f1-score   support

         0.0       0.70      0.12      0.20      1706
         1.0       0.91      0.99      0.95     14765

    accuracy                           0.90     16471
   macro avg       0.80      0.56      0.57     16471
weighted avg       0.89      0.90      0.87     16471

ROC-AUC Score: 0.5551859158072007


/tmp/ipykernel_40662/1837066132.py:720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_df['is_promo'] = xgb_model.predict(X_predict)
/tmp/ipykernel_40662/1837066132.py:764: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result = data.groupby('ean_global_channel').apply(update_subtactics_and_price_).reset_index(drop=True)


Interpolation step is done


In [9]:
train_set1, test_set1, prediction_len = import_true_promo(bq_client, 10, 12, 50,channel='Offline', fill_discontinuity=False, keep_non_promo=True)

/home/user/miniconda/envs/vertex/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


number of products before preprocessing 2412
How many ean_global_channel_type: 1928
final data product (if changed we remove discontinuity) 1873
prediction length: 12


ValueError: too many values to unpack (expected 3)